# GPDM Experiments
---

In [ ]:
import GPflow
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
from GPflow.gplvm import GPLVM
from gpdm import GPDM
import matplotlib.pyplot as plt
from bcgplvm import BCGPLVM
from GPflow import kernels, ekernels
from GPflow.plotting import plotLatent

%matplotlib inline

In [ ]:
# set parameters for experiments
np.random.seed(42)

# throw error if quadrature is used for kernel expectations
GPflow.settings.numerics.quadrature = 'error'  

## Experiment with Linear Dynamical System
---

In [ ]:
# d = 2, D = 10, linear dyn, linear map
d = 2
D = 10
nSamples = 100
noiseVar = 0.01
t = np.linspace(0.0,5.0,num=nSamples,endpoint=True)

# generate latent points based on LDS x(t) = [[-1 0],[0 -2]]*x(t-1)
X = np.exp(np.asarray([-t,-2*t])).T
X = X - X.mean(axis=0)

# generate high dimensional observation data
Y1 = np.matmul(np.random.randn(D,d),X.T).T + np.random.randn(nSamples,D)*np.sqrt(noiseVar)
Y2 = np.sin(np.matmul(np.random.randn(D,d),X.T).T)*np.tanh(np.matmul(np.random.randn(D,d),X.T).T) + np.random.randn(nSamples,D)*np.sqrt(noiseVar)

In [ ]:
# visualize original latent data
fig = plt.figure(figsize=(10,6))
plt.scatter(X[:,0], X[:,1], color='k')
plt.title('Original Data')

In [ ]:
# train GPLVM, GPDM on given data
m1 = GPLVM(Y1, d, kern=kernels.Linear(d))
m1.likelihood.variance = noiseVar
_ = m1.optimize(disp=True, maxiter=3000)

m2 = GPDM(Y1, d, map_kern=kernels.Linear(d), dyn_kern=kernels.Linear(d))
m2.likelihood.variance = noiseVar
m2.dyn_likelihood.variance = 1e-5
_ = m2.optimize(disp=True, maxiter=3000)

# visualize original latent data
fig,ax = plt.subplots(1,2,figsize=(10,6))

ax[0].scatter(m1.X.value[:,0], m1.X.value[:,1], color='k')
ax[0].set_title('GPLVM Latent Space')
ax[1].scatter(m2.X.value[:,0], m2.X.value[:,1], color='k')
ax[1].set_title('GPDM Latent Space')

plt.suptitle('Linear Mapping')

In [ ]:
# train GPLVM, GPDM on given data
m1 = GPLVM(Y2, d, kern=kernels.RBF(d))
m1.likelihood.variance = noiseVar
_ = m1.optimize(disp=True, maxiter=3000)

m2 = GPDM(Y2, d, map_kern=kernels.RBF(d), dyn_kern=kernels.Linear(d))
m2.likelihood.variance = noiseVar
m2.dyn_likelihood.variance = 1e-5
_ = m2.optimize(disp=True, maxiter=3000)

# visualize original latent data
fig,ax = plt.subplots(1,2,figsize=(10,6))

ax[0].scatter(m1.X.value[:,0], m1.X.value[:,1], color='k')
ax[0].set_title('GPLVM Latent Space')
ax[1].scatter(m2.X.value[:,0], m2.X.value[:,1], color='k')
ax[1].set_title('GPDM Latent Space')

plt.suptitle('Nonlinear Mapping')